# Time series analysis of hyp3 InSAR products by MintPy

This notebook shows how to do time-series analysis with HyP3 InSAR product by MintPy. We assume you have already got the hyp3 InSAR products somewhere. This steps for the analysis are: clip the hyp3 INSAR product, define the config.txt file, run the time series analysis, and display the results. We use 2019 Ridgecrest Earthquake, CA (https://earthquake.usgs.gov/storymap/index-ridgecrest.html) as the example. The example hyp3 INSAR data are at https://jzhu-hyp3-dev.s3.us-west-2.amazonaws.com/hyp3-mintpy-example/2019_ridgecrest.zip. As far as how to produce the hyp3 INSAR product, we provide the detail steps in the tutorial(https://github.com/ASFHyP3/hyp3-docs/blob/develop/docs/tutorials/hyp3_insar_stack_for_ts_analysis.ipynb). 


## 0. Initial setup of the notebook

To run this notebook, you'll need a conda environment with the required dependencies. You can set up a new environment (recommended) and run the jupyter server like:

conda create -n hyp3-mintpy python=3.8 asf_search hyp3_sdk "mintpy>=1.3.2" pandas jupyter ipympl

To make you conda env accessible in the jupyter notebook, you need to do:

conda activate hyp3-mintpy
conda install -c conda-forge tensorflow
conda install -c anaconda ipykernel
python -m ipykernel install --user --name=hyp3-mintpy

To run your notebook, just:

conda activate hyp3-mintpy
jupyter notebook smallbaselineApp_hyp3_new.ipynb

For your convinience, we provide the ERA5 data at https://jzhu-hyp3-dev.s3.us-west-2.amazonaws.com/hyp3-mintpy-example/2019_ridgecrest_era5_data.zip, you can download the and unzip the file to the 'your_weather_dir' directory on you local machine, and setup the environment variable. e.g. export WEATHER_DIR='your_weather_dir'.


In [ ]:
from pathlib import Path

### Define the parameters and create directories

In [ ]:
from pathlib import Path

project_name = 'Ridgecrest'

work_dir = Path.cwd() / project_name

data_dir = work_dir / 'data'

data_dir.mkdir(parents=True, exist_ok=True)


## 1. Run MintPy routine workflow `smallbaselineApp.py`

### 1.1 Load the hyp3 InSAR data

The example dataset is from 2019 Ridgecrest Earthquake, CA. The dataset can be obtained through either downloading from the stagged server or producing with hyp3-sdk. Here we provide the sample dataset at  

In [ ]:
file = '2019_ridgecrest.zip'

file_url = f'https://jzhu-hyp3-dev.s3.us-west-2.amazonaws.com/hyp3-mintpy-example/{file}'


In [ ]:
!wget {file_url} -P {data_dir}

In [ ]:
print(f'downloaded file is {data_dir}/{file}')

In [ ]:
import os
import zipfile

def unzip_files(zip_file, data_dir):
    if os.path.isfile(zip_file):
        with zipfile.ZipFile(zip_file, 'r') as fzip:
            fzip.extractall(data_dir)


In [ ]:
unzip_files(f'{data_dir}/{file}', data_dir)

### 1.2 Cut geotiff files for mintpy analysis

### get the minumum overlap of the files

In [ ]:
from pathlib import Path
from typing import List, Union
from osgeo import gdal


def get_minimum_overlap(filelist: List[Union[str, Path]]) -> List[float]:
    """Get the minimum overlap of the geotiff files in the filelist.
    
    Arg:
        filelist: a list of geotiff file names. The file names can be strings or Path objects.
    
    Returns:
         [ulx, uly, lrx, lry], a list which includes the upper-left x, upper-left y, lower-right x, 
         and lower-right y.
    """
    corners = [gdal.Info(str(dem), format='json')['cornerCoordinates'] for dem in filelist]

    ulx = max(corner['upperLeft'][0] for corner in corners)
    uly = min(corner['upperLeft'][1] for corner in corners)
    lrx = min(corner['lowerRight'][0] for corner in corners)
    lry = max(corner['lowerRight'][1] for corner in corners)
    return [ulx, uly, lrx, lry]


In [ ]:
files = data_dir.glob('*/*_dem.tif')

overlap = get_minimum_overlap(files)

### clip the files with overlap

In [ ]:
from pathlib import Path, Path
from typing import List, Union

def clip_hyp3_products_to_minimum_overlap(data_dir: Union[str, Path], overlap: List[float]) -> None:
    """Clip all geotiff files in the directory with the overlap.
    
    Args:
        data_dir:
            name of a directory which includes the geotiff files.
        overlap:
            a list which includes the upper-left x, upper-left y, lower-right-x, and lower-tight y.

    Returns: None
    """

    files_for_mintpy = ['_water_mask.tif', '_corr.tif', '_unw_phase.tif', '_dem.tif', '_lv_theta.tif', '_lv_phi.tif']

    for extension in files_for_mintpy:

        for file in data_dir.rglob(f'*{extension}'):

            dst_file = file.parent / f'{file.stem}_clipped{file.suffix}'

            gdal.Translate(destName=str(dst_file), srcDS=str(file), projWin=overlap)


In [ ]:
clip_hyp3_products_to_minimum_overlap(data_dir, overlap)

### 1.3 Prepare the template file

In [ ]:
mintpy_config = work_dir / 'mintpy_config.txt'
mintpy_config.write_text(
f"""
mintpy.load.processor        = hyp3
##---------interferogram datasets:
mintpy.load.unwFile          = {data_dir}/*/*_unw_phase_clipped.tif
mintpy.load.corFile          = {data_dir}/*/*_corr_clipped.tif
##---------geometry datasets:
mintpy.load.demFile          = {data_dir}/*/*_dem_clipped.tif
mintpy.load.incAngleFile     = {data_dir}/*/*_lv_theta_clipped.tif
mintpy.load.azAngleFile      = {data_dir}/*/*_lv_phi_clipped.tif
mintpy.load.waterMaskFile    = {data_dir}/*/*_water_mask_clipped.tif
""")

### 1.4 Run Time-series Analysis application

In [ ]:
! smallbaselineApp.py --work-dir {work_dir}  {mintpy_config}

## 2. Display the analysis results

There are a few scripts used to display the analysis results. There are in the MINTPY_HOME/mintpy. Here we show two majoy disaply scripts.

In [ ]:
%matplotlib widget
from mintpy import view, tsview

In [ ]:
view.main([f'{work_dir}/velocity.h5'])

In [ ]:
tsview.main([f'{work_dir}/timeseries.h5'])